# 🎯 TrustPilot Monitor Inteligente de Opinión
## 💎 Herramienta de Inteligencia Comercial Unificada y Mejorada

Este notebook representa la unión definitiva de los prototipos iniciales, optimizado para ofrecer una solución robusta, escalable y profesional para el análisis de la reputación digital.

### 👤 Créditos y Autoría:
- **Base del Proyecto**: [Juanes] (MONITOR_INTELIGENCIA_OPINION / Prototipos Iniciales)
- **Mejoras y Nuevos Módulos**: [Rubén] (Búsqueda dinámica, Modo Stealth, IA Híbrida, Dashboard Temporal)

### 🔬 Fases del Monitor:
1.  📥 **ADQUISICIÓN DE DATOS**: Scraping inteligente con rotación de identidad y búsqueda dinámica. [Base: Juanes | Mejoras: Rubén]
2.  🧹 **PREPROCESAMIENTO NLP**: Limpieza profunda y tokenización de textos en español. [Autor: Juanes]
3.  💎 **ANÁLISIS DE SENTIMIENTO**: Motor híbrido diccionario + IA avanzada. [Base: Juanes | Refuerzo AI: Rubén]
4.  📊 **DASHBOARD BI**: Visualización de impacto para toma de decisiones. [Visuales: Juanes | Eje Temporal: Rubén]

In [ ]:
# =============================================================================
# FASE 0: PREPARACIÓN DEL ENTORNO
# [Mejora Compatibilidad Rubén / Original Juanes]
# =============================================================================
import sys
import subprocess

print("🔧 Inicializando entorno del monitor...")

def setup_environment():
    # Listado de librerías esenciales unificadas
    libs = [
        "requests", "beautifulsoup4", "lxml", "nltk", "textblob", 
        "googletrans==4.0.0-rc1", "wordcloud", "matplotlib", 
        "seaborn", "plotly", "fake-useragent", "pandas", 
        "numpy", "regex", "tqdm", "spacy"
    ]
    
    for lib in libs:
        try:
            # [Rubén] Uso de sys.executable para garantizar instalación en el kernel activo
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        except:
            pass
    
    # Descarga de recursos NLP
    import nltk
    nltk.download(['punkt', 'stopwords', 'punkt_tab'], quiet=True)
    
    try:
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_sm", "-q"])
    except:
        pass

setup_environment()
print("✅ Entorno listo para procesamiento.")

In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import time
import random
from datetime import datetime
from typing import List, Dict, Optional
from collections import Counter
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from textblob import TextBlob
from googletrans import Translator
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8')
print("📚 Librerías de análisis cargadas.")

# 📥 FASE 1: SCRAPER INTELIGENTE DE TRUSTPILOT
### [Base: Juanes | Mejoras Rubén: Búsqueda y Modo Stealth]

In [ ]:
class TrustpilotScraper:
    def __init__(self, business_query: str = None, max_pages: int = 5):
        self.query = business_query
        self.max_pages = max_pages
        self.ua = UserAgent() # [Rubén] Rotación de identidad
        self.session = requests.Session()
        self.base_url = "https://es.trustpilot.com"
        self.target_url = None
        self.reviews_data = []

    def _get_headers(self):
        """[Rubén] Modo Stealth: Encabezados realistas aleatorios."""
        return {
            'User-Agent': self.ua.random,
            'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
            'Referer': 'https://www.google.com/'
        }

    def search_business(self) -> bool:
        """[Rubén] Búsqueda automática por palabra clave."""
        if not self.query: return False
        if "trustpilot.com/review/" in self.query:
            self.target_url = self.query
            return True

        print(f"🔍 Buscando: '{self.query}'...")
        try:
            s_url = f"{self.base_url}/search?query={self.query.replace(' ', '+')}"
            res = self.session.get(s_url, headers=self._get_headers())
            soup = BeautifulSoup(res.content, 'html.parser')
            link = soup.select_one('a[data-business-unit-card-link="true"]')
            if link: 
                self.target_url = self.base_url + link['href']
                print(f"✅ Encontrado: {self.target_url}")
                return True
        except: pass
        return False

    def safe_extract(self, element, select_list: List[str], attr: str = None) -> str:
        """[Juanes] Lógica resiliente ante cambios de HTML."""
        for selector in select_list:
            found = element.select_one(selector)
            if found:
                return found.get(attr, "") if attr else found.get_text(strip=True)
        return ""

    def run(self) -> pd.DataFrame:
        if not self.target_url and not self.search_business(): return pd.DataFrame()
        
        for page in range(1, self.max_pages + 1):
            print(f"📄 Página {page}...")
            time.sleep(random.uniform(2, 4)) # [Rubén] Delay antidetect
            try:
                res = self.session.get(f"{self.target_url}?page={page}", headers=self._get_headers())
                soup = BeautifulSoup(res.content, 'html.parser')
                cards = soup.select('article[data-service-review-card-paper="true"]')
                
                for card in cards:
                    text = self.safe_extract(card, ['p[data-service-review-text-typography="true"]', 'p[data-review-content-typography="true"]'])
                    if not text: continue
                    
                    rating_alt = self.safe_extract(card, ['.styles_reviewHeader__iU9_n img'], 'alt')
                    rating = re.search(r'\d', rating_alt).group() if rating_alt else "0"
                    date_iso = self.safe_extract(card, ['time'], 'datetime')
                    
                    self.reviews_data.append({
                        'texto': text,
                        'puntuacion': int(rating),
                        'fecha': date_iso[:10] if date_iso else datetime.now().strftime('%Y-%m-%d')
                    })
            except: break
        return pd.DataFrame(self.reviews_data)

In [ ]:
# [Rubén] Ejecución del Scraper
empresa = input("Ingrese nombre de empresa o URL de Trustpilot: ") or "Amazon Spain"
df_raw = TrustpilotScraper(empresa, max_pages=3).run()

if df_raw.empty:
    # Dataset de cortesía para pruebas rápidas si no hay internet o bloqueo
    df_raw = pd.DataFrame({'texto': ["Muy bueno", "Mal servicio", "Perfecto", "No llegó"], 'puntuacion': [5, 1, 5, 1], 'fecha': ["2024-01-01"]*4})

print(f"📌 {len(df_raw)} reseñas capturadas.")

# 🧹 FASE 2: PREPROCESAMIENTO NLP
### [Autor Principal: Juanes]

In [ ]:
class TextCleaner:
    def __init__(self):
        self.stop = set(stopwords.words('spanish'))
        self.stop.update(['amazon', 'servicio', 'producto', 'pedido', 'envío'])

    def clean(self, text):
        # [Juanes] Normalización y eliminación de ruido
        text = text.lower()
        text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
        tokens = word_tokenize(text)
        return ' '.join([t for t in tokens if t not in self.stop and len(t) > 2])

cleaner = TextCleaner()
df_proc = df_raw.copy()
df_proc['texto_limpio'] = df_proc['texto'].apply(cleaner.clean)
print("🧹 Textos procesados y limpios.")

# 💎 FASE 3: ANÁLISIS DE SENTIMIENTO
### [Base Juanes | Mejora AI Rubén]

In [ ]:
class SentimentEngine:
    def __init__(self):
        # [Juanes] Diccionarios de referencia ràpida
        self.pos = {'excelente', 'perfecto', 'bueno', 'rápido', 'recomiendo'}
        self.neg = {'malo', 'pésimo', 'lento', 'estafa', 'horrible'}
        self.trans = Translator()

    def analyze(self, text):
        words = set(text.split())
        p_score = len(words & self.pos)
        n_score = len(words & self.neg)
        
        # [Rubén] Fallback a IA avanzada si el texto es descriptivo
        if len(text) > 50 and p_score == n_score:
            try:
                en_text = self.trans.translate(text, dest='en').text
                score = TextBlob(en_text).sentiment.polarity
                return 'positivo' if score > 0 else ('negativo' if score < 0 else 'neutral')
            except: pass
            
        return 'positivo' if p_score > n_score else ('negativo' if n_score > p_score else 'neutral')

engine = SentimentEngine()
df_proc['sentimiento'] = df_proc['texto_limpio'].apply(engine.analyze)
print(df_proc['sentimiento'].value_counts(normalize=True))

# 📊 FASE 4: VISUALIZACIÓN GRÁFICA (BI)
### [Diseño Juanes | Rubén: Visión Temporal]

In [ ]:
plt.figure(figsize=(15, 10))

# 1. Distribución Sentimiento [Juanes]
plt.subplot(2, 2, 1)
sns.countplot(data=df_proc, x='sentimiento', palette='viridis')
plt.title("Distribución de Sentimiento [Juanes]")

# 2. Nube de Palabras [Juanes]
plt.subplot(2, 2, 2)
wc = WordCloud(background_color='white').generate(' '.join(df_proc['texto_limpio']))
plt.imshow(wc)
plt.axis('off')
plt.title("Nube de Temas [Juanes]")

# 3. Tendencia Temporal [Rubén]
plt.subplot(2, 2, 3)
df_proc['fecha'] = pd.to_datetime(df_proc['fecha'])
df_proc.groupby('fecha')['puntuacion'].mean().plot(marker='o', color='red')
plt.title("Evolución de Satisfacción [Rubén]")

# 4. Consistencia IA vs Estrellas [Rubén]
plt.subplot(2, 2, 4)
sns.boxplot(data=df_proc, x='sentimiento', y='puntuacion')
plt.title("Consistencia IA vs Rating [Rubén]")

plt.tight_layout()
plt.show()